# Crear la clase  Target

## Ejercicio 1

Cree el archivo competencia_01.csv, usando el competencia_01_crudo.csv con una variable adicional llamada clase_ternaria, que contenga las categorías **CONTINUA, BAJA+1 y BAJA+2**

Ayudita
Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**.

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction) Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

Couldn't find program: 'bash'


In [2]:
%pip install duckdb jupysql duckdb-engine


Note: you may need to restart the kernel to use updated packages.


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb**

In [2]:

import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///mi_base.duckdb


Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
select 'hola mundo'

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla pasando su ubicación a través de la variable `dataset_path`:

In [4]:
# dataset_path = '/home/aleb/DMEyF/2024/datos/'
dataset_file = 'competencia_01_crudo.csv'

In [ ]:
%%sql
create or replace table competencia_01_crudo as
select *
from read_csv_auto("{{dataset_file}}")

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [11]:
%sql select * from competencia_01_crudo limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,0.0,4,0.00,-20833.47,0.0,2814,6743.38,6,0,9559.95
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,0.0,11,13267.49,-9662.51,0.0,5541,9419.90,8,0,1900.26
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,0.0,4,0.00,-4692.00,0.0,577,2821.32,2,0,6885.51
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,0.0,25,23800.17,-20349.22,0.0,7916,23509.71,8,0,1219.92
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,0.0,4,0.00,-88178.37,0.0,1958,20397.54,8,0,2627.52


In [10]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_01_crudo
group by foto_mes

,foto_mes,cantidad
0,202101,161527
1,202102,162155
2,202103,163109
3,202104,163418
4,202105,163917
5,202106,164313


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide.

In [13]:
%%sql
create or replace table competencia_01 as (
    select numero_de_cliente,
           foto_mes,
           case when foto_mes_1 is null then 'BAJA+1'
                when foto_mes_2 is null then 'BAJA+2'
                else 'CONTINUA'
           end as target
    from (
        select numero_de_cliente,
               foto_mes,
               LEAD(foto_mes, 1, NULL) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) foto_mes_1,
               LEAD(foto_mes, 2, NULL) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) foto_mes_2
        from competencia_01_crudo
    ) as a
    where foto_mes <= 202104
)


,Success


In [21]:
%sql select * from competencia_01 limit 50

,numero_de_cliente,foto_mes,target
0,1061181994,202101,CONTINUA
1,1061181994,202102,CONTINUA
2,1061181994,202103,CONTINUA
3,1061181994,202104,CONTINUA
4,1061265641,202101,CONTINUA
5,1061265641,202102,CONTINUA
6,1061265641,202103,CONTINUA
7,1061265641,202104,CONTINUA
8,1061476034,202101,CONTINUA
9,1061476034,202102,CONTINUA


## Ejercicio 1.1
* ¿Cuál es la nominalidad de cada clase?


In [15]:
%%sql
PIVOT competencia_01
on target
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,616,824,160087
1,202102,830,1027,160298
2,202103,1034,953,161122
3,202104,954,1131,161333


* ¿Cuál es la proporción del target?

## Guardar tabla en .csv

Para guardar a un `.csv` simplemente debe ejecutar la siguiente sentencia

In [25]:
%sql COPY competencia_01 TO 'competencia_01.csv' (FORMAT CSV, HEADER)

,Success
